In [0]:
import time
import datetime
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import inspect

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [0]:
import random


In [0]:
import datetime as dt
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
def CountRowNum(df,Column):
  return df.agg(count(Column).alias("cnt_"+Column),countDistinct(Column).alias("dcnt_"+Column))
def CountColNum(df):
  return len(df.columns)

## for tansformation

In [0]:
def get_dummy(df,id_lst):
  catg_casewhen_lst = []
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect() if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect() if i.col_name not in id_lst]
  print("non_catg_column: \n",non_catg_column)

  # get distinct data content from each column
  for c in catg_column: 
    distinct_lst = [row[0] for row in df.selectExpr(
      """lower(replace(replace(replace(replace(replace(replace(replace({c},'-','_'),'/','_'),'&','_'),'!','_'),'#','_'),' ','_'),'.','_')) as {c}""".format(c= c)
                                                         ).distinct().collect()]
    # trainsform data into dummy 
    for i in range(len(distinct_lst)):
      catg_casewhen_lst.append("""case when lower(replace(replace(replace(replace(replace(replace({c},'-','_'),'/','_'),'&','_'),'!','_'),'#','_'),' ','_')) = '{x}' then 1 else 0 end as {c}_{x}""".format(c = c, x = distinct_lst[i]))
#   return(catg_casewhen_lst)

# produce dummy data 
  df_dummy = df.selectExpr(*id_lst,*non_catg_column,*catg_casewhen_lst)
  return(df_dummy)

In [0]:
def get_cnt_from_catg_data(df,id_lst):
  catg_casewhen_lst = []
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect() if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect()if i.col_name not in id_lst]
  print("non_catg_column: \n",non_catg_column)
  
  #print('1')
  #df_catg_all = df.groupBy(catg_column[0]).agg(countDistinct(col(id_)).alias('id_cnts'))
  
  
  for c in range(len(catg_column)):
    #print(c)
    df_catg = df.groupBy(catg_column[c]).agg(countDistinct(col(id_)).alias('id_cnts')) 
    display(df_catg)

  return()

In [0]:
#percentile_approx: https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.percentile_approx.html
def summary_for_numeric_data(df,id_lst):
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect() if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect() if i.col_name not in id_lst]
  
  
  print("non_catg_column: \n",non_catg_column)
  
  #print('1')
  #df_catg_all = df.groupBy(catg_column[0]).agg(countDistinct(col(id_)).alias('id_cnts'))
  
  
  for c in range(len(non_catg_column)):
    print(non_catg_column[c],":")
    df_summary = df.agg(round(max(col(non_catg_column[c])),2).alias('max'),
                        round(min(col(non_catg_column[c])),2).alias('min'),
                        round(mean(col(non_catg_column[c])),2).alias('mean')
                       ) 
    # df_summary.withColumn("Q1",lit(winback.select(percentile_approx("P6M_AVG_INV_AMT", 0.25, 1000000).alias('Q1')).collect()[0].Q1))
    df_summary = df_summary.withColumn("Q1",lit(df.select(percentile_approx(non_catg_column[c], 0.25, 1000000).alias('Q1')).collect()[0].Q1))
    df_summary = df_summary.withColumn("Q2",lit(df.select(percentile_approx(non_catg_column[c], 0.5, 1000000).alias('Q2')).collect()[0].Q2))
    df_summary = df_summary.withColumn("Q3",lit(df.select(percentile_approx(non_catg_column[c], 0.75, 1000000).alias('Q3')).collect()[0].Q3))
    display(df_summary)
  return()

In [0]:
#percentile_approx: https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.percentile_approx.html
def select_for_numeric_data(df,id_lst):
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect() if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect() if i.col_name not in id_lst]
  
  
  print("non_catg_column: \n",non_catg_column)
  
  #print('1')
  #df_catg_all = df.groupBy(catg_column[0]).agg(countDistinct(col(id_)).alias('id_cnts'))
  
  
  for c in range(len(non_catg_column)):
    display(df.select(non_catg_column[c]))
  return()

In [0]:
#percentile_approx: https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.percentile_approx.html
def countnull_for_numeric_data(df,id_lst):
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect() if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect() if i.col_name not in id_lst]
  
  
  print("non_catg_column: \n",non_catg_column)
  
  #print('1')
  #df_catg_all = df.groupBy(catg_column[0]).agg(countDistinct(col(id_)).alias('id_cnts'))
  
  
  for c in range(len(non_catg_column)):
    display(df.select(count(when(isnan(non_catg_column[c]) | col(non_catg_column[c]).isNull(), non_catg_column[c])).alias(non_catg_column[c])))
  return()

In [0]:
def fillna_for_numeric_data(df,id_lst, fill_value):
  catg_casewhen_lst = []
  
  descri_table = spark.createDataFrame(df.dtypes,['col_name','data_type'])
  descri_table.where("data_type = 'string' ")

  #get column names by datatype
  catg_column = [i.col_name for i in descri_table.where("data_type = 'string' ").collect()if i.col_name not in id_lst]
  print("catg_column: \n", catg_column)

  non_catg_column = [i.col_name for i in descri_table.where("data_type in ('date','bigint','int','double','float') ").collect()if i.col_name not in id_lst]
  print("non_catg_column: \n",non_catg_column)
  
  #print('1')
  #df_catg_all = df.groupBy(catg_column[0]).agg(countDistinct(col(id_)).alias('id_cnts'))
  
  
  df_fillna = df.fillna(fill_value, subset= non_catg_column)

  return(df_fillna)

## for pandas

In [0]:
def convert_datatype_for_pandas(full_data):
  #convert decimal to float
  select_expr = [col(c).cast("float") if ("decimal" in t) else col(c) for c, t in full_data.dtypes]
  full_data = full_data.select(*select_expr)
  #convert double to float
  select_expr = [col(c).cast("float") if ("double" in t) else col(c) for c, t in full_data.dtypes]
  full_data = full_data.select(*select_expr)

  #convert boolean to int
  select_expr = [col(c).cast("int") if ("boolean" in t) else col(c) for c, t in full_data.dtypes]
  full_data = full_data.select(*select_expr)
  return full_data.printSchema()

## for model evaluation

In [0]:
def eval_pct_qcut10(y_proba, y_test):
  df_proba = pd.DataFrame({"predict_proba":y_proba[:,1], "actual":y_test})
  df_proba['rank'] = df_proba['predict_proba'].rank(method='first',ascending=False)
  df_proba['decile'] = pd.qcut(df_proba['rank'].values, 10).codes
  df_proba = df_proba.sort_values(by=['decile'])
  groups= df_proba.groupby("decile")
  pct = groups["actual"].sum()/groups.size()
  test_base_rate = (df_proba["actual"].sum()/len(df_proba["actual"]))
  top10 = pct[0]
  lift = top10 / test_base_rate
  return lift, test_base_rate, pct